In [1]:
import os
import sys
import datetime
import numpy as np

In [2]:
import pandas as pd
# import s3fs
pd.set_option('display.max_colwidth', -1)
sys.path.append(os.path.join(os.getcwd(), '..'))
from aws_utils import read_csv_as_dataframe

In [3]:
today = datetime.datetime.now()
yesterday = today - datetime.timedelta(days=1)
yesterday = yesterday.strftime("%Y-%m-%d")
yesterday

'2019-11-02'

# Read and clean input

In [4]:
def read_data_apartments(website, location):
    BUCKET = f's3://data-apartments/production/{website}/{location}/'
    df = read_csv_as_dataframe(bucket_path=BUCKET,
                               filename=f'{website}_db.csv',
                               timestamp_column_name='downloaded')
    df['website'] = website
    df['city'] = location
    return df

df = (read_data_apartments('olx', 'warszawa')
          .append(read_data_apartments('otodom', 'warszawa'), sort=False)
          .append(read_data_apartments('gumtree', 'warszawa'), sort=False))


/home/filip/.local/share/virtualenvs/otodom-YV8_LmfU/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (4,5,6,7,9,10,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/filip/.local/share/virtualenvs/otodom-YV8_LmfU/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (0,3,5,6,7,9,11,12,13,14,15,16,17,18,19,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/filip/.local/share/virtualenvs/otodom-YV8_LmfU/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (4,6,7,10,11,12,13,14,15,16,17,18,19,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [95]:
d = df.copy()
# Remove mistakes from collection system
d.query('rooms != "rooms"', inplace=True)
# Remove instances where coordinates were not recorded
d.drop('tracking_id', axis=1, inplace=True)
d.dropna(subset=['latitude', 'longitude', 'owner', 'rooms', 'area'], inplace=True)
d.query('downloaded > @yesterday', inplace=True)
# Drop duplicates
d.drop_duplicates(subset=['item_id', 'website'], inplace=True)
# Convert to numeric columns
num_cols = ['area', 'price_per_m', 'price', 'latitude', 'longitude']
for col in num_cols:
    d[col] = pd.to_numeric(d[col])

## Map rooms to numerical value

In [96]:
d.rooms = pd.to_numeric(d.rooms, errors='ignore', downcast='integer')
rooms = {
    "Kawalerka lub garsoniera": 1,
    "4 pokoje": 4,
    "6 lub więcej pokoi": 6,
    "5 pokoi": 5,
    "2 pokoje": 2,
    "3 pokoje": 3,
}
d.rooms.replace(rooms, inplace=True)
d.rooms = d.rooms.astype(int)

## Organize owner column

In [97]:
owner = {
    'Osoby prywatnej': 'private',
    'Biuro / Deweloper': 'agency',
    'Oferta biura nieruchomości': 'agency',
    'Oferta prywatna': 'private',
    'Agencja': 'agency',
    'Właściciel': 'agency',
}

d.owner = d.owner.map(owner)
d.owner.unique()

array(['private', 'agency'], dtype=object)

## Calculate distance from center in degrees

In [98]:
center_latitude = 52.229719
center_longitude = 21.011381
d['latitude_from_center'] = (center_latitude - d['latitude']).abs()
d['longitude_from_center'] = (center_longitude - d['longitude']).abs()
# distance from center
d['distance_from_center'] = np.sqrt((d['latitude_from_center']**2+d['longitude_from_center']**2))*111

In [99]:
def create_distance_clusters(x):
    if x <= 2:
        return "city_centre"
    elif 2 < x <= 5:
        return "close_city_centre"
    elif 5 < x <= 10:
        return "downtown"
    else:
        return "suburbs"
    
d['location_name'] = d['distance_from_center'].apply(create_distance_clusters)

In [100]:
price_distr = (d[['price_per_m', 'location_name']].groupby('location_name')
                                                  .agg(['mean', 'std']))
price_distr.columns = price_distr.columns.droplevel(0)
price_distr.reset_index(inplace=True)
# Merge with results
d = d.merge(price_distr, on='location_name', how='inner')

## Map districts

In [45]:
districts = ['Mokotów', 'Ursus', 'Bemowo', 'Śródmieście', 'Wilanów', 'Targówek', 'Ochota', 'Białołęka', 'Ursynów',
             'Włochy', 'Wawer', 'Wola', 'Bielany', 'Wesoła', 'Żoliborz', 'Rembertów', 'Praga-Północ', 'Praga-Południe']

districts_map = {
    'Praga Południe': 'Praga-Południe',
    'Praga Północ': 'Praga-Północ',
    'Centrum': 'Śródmieście',
}

d.district.replace(districts_map, inplace=True)
d.query('district in @districts', inplace=True)
d.shape

(1919, 51)

### Encode districts

In [46]:
d = pd.concat([d, pd.get_dummies(d['district'], prefix='district')], axis=1)
d = pd.concat([d, pd.get_dummies(d['owner'], prefix='owner')], axis=1)

## Remove outliers/ quirks

In [101]:
d.query('price < 30000000', inplace=True)
d.query('price_per_m < 1000000', inplace=True)
d.query('area < 1000', inplace=True)

# Select offers below standard deviation

In [102]:
d['threshold'] = d['mean'] - d['std']
d_offers = d.query('price_per_m < threshold')

In [103]:
d_selected_offer_ids = (d_offers[['item_id', 'location_name']]
                             .groupby('location_name')
                             .apply(lambda x: x.sample(2))
                             .item_id
                             .values)
d_offers.query('item_id in @d_selected_offer_ids')

,downloaded,url,short_description,district,area,rooms,price_per_m,price,owner,item_id,...,longitude,website,city,latitude_from_center,longitude_from_center,distance_from_center,location_name,mean,std,threshold
327,2019-11-02 00:00:00,https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/praga-poludnie/mieszkanie-warszawa-praga-poludnie-139-7m2-nr-hhn+ms+8711+20/1006211109070911381140509,Mieszkanie Warszawa Praga Południe 139.7m2 (nr: HHN-MS-8711-20),Praga Południe,140.0,5,7135.714286,999000,agency,1006211109070911381140509,...,21.086269,gumtree,warszawa,0.000344,0.074888,8.312656,downtown,10904.185380,2770.134360,8134.051021
650,2019-11-03 00:00:00,https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/wlochy/wlochy-dom-632-m2-na-firme-2014/1006214704560911543980109,"Włochy, dom 632 m2, na firmę, 2014",Włochy,632.0,6,6724.683544,4250000,agency,1006214704560911543980109,...,20.959474,gumtree,warszawa,0.046171,0.051907,7.711184,downtown,10904.185380,2770.134360,8134.051021
792,2019-11-02 00:00:00,https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/ochota/mieszkanie-warszawa-ochota-132m2-nr-hhn+ms+4366+6/1006211106720911381140509,Mieszkanie Warszawa Ochota 132m2 (nr: HHN-MS-4366-6),Ochota,132.0,5,6590.909091,870000,agency,1006211106720911381140509,...,20.973807,gumtree,warszawa,0.022782,0.037574,4.877468,close_city_centre,11360.600110,2778.264764,8582.335347
816,2019-11-02 00:00:00,https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/praga-polnoc/mieszkanie-warszawa-praga-polnoc-68m2-nr-hhn+ms+8999+30/1006211106130911381140509,Mieszkanie Warszawa Praga Północ 68m2 (nr: HHN-MS-8999-30),Praga Północ,68.0,3,8088.235294,550000,agency,1006211106130911381140509,...,21.030906,gumtree,warszawa,0.030183,0.019525,3.990198,close_city_centre,11360.600110,2778.264764,8582.335347
1171,2019-11-03 00:00:00,https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/bielany/sprzedam-bezposrednio-dom-w-zabudowie-blizniaczej-cena-do-negocjacji/1004814057140910479525809,SPRZEDAM BEZPOŚREDNIO DOM W ZABUDOWIE BLIŻNIACZEJ (cena do negocjacji),Bielany,185.0,5,4864.864865,900000,agency,1004814057140910479525809,...,20.927347,gumtree,warszawa,0.066610,0.084034,11.902702,suburbs,7829.590279,2745.051023,5084.539256
1199,2019-11-03 00:00:00,https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/wawer/wolnostojcy-6+pok-267-6-m2-dzialka-500-m2/1006214826240911543980109,"Wolnostojcy, 6-pok., 267, 6 m2, działka 500 m2",Wawer,437.0,6,2116.704805,925000,agency,1006214826240911543980109,...,21.189887,gumtree,warszawa,0.029861,0.178506,20.089489,suburbs,7829.590279,2745.051023,5084.539256
1501,2019-11-02 00:00:00,https://www.olx.pl/oferta/lodz-manufaktura-3-pak-kawalerek-roi-od-7-3-2x-parking-w-cenie-CID3-IDA1JaV.html#433628d34c;promoted,Łódź-Manufaktura! 3-pak Kawalerek! ROI od 7.3%! ! 2x parking w cenie!,Śródmieście,68.0,1,8970.000000,610000,private,532360082,...,21.017197,olx,warszawa,0.006305,0.005816,0.952203,city_centre,15510.501497,4419.847531,11090.653966
1727,2019-11-03 00:00:00,https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/srodmiescie/mieszkanie-na-srodmiesciu-ul-inflancka/1006214341380912450050009,Mieszkanie na śródmieściu ul. Inflancka,Śródmieście,65.0,3,11076.923077,720000,agency,1006214341380912450050009,...,21.001572,gumtree,warszawa,0.001828,0.009809,1.107545,city_centre,15510.501497,4419.847531,11090.653966


In [106]:
d_selected_offers.tolist()

['556885983',
 '494621221',
 '1006214704680911543980109',
 '1006214914090911543980109',
 '1006214814190911543980109',
 '1006214770560911543980109',
 '1006213911960911381140509',
 '1006214593670911543980109']

# Start clustering

## Determine number of clusters

In [ ]:
# Select apropriate columns
columns = [c for c in d.columns if 'owner_' in c]
X = d.loc[:, columns + ['area', 'rooms', 'price', 'latitude_from_center', 'longitude_from_center']]

In [ ]:
inertias = []
clus_num = []
for k in range(10,30):
    scaler = StandardScaler()
    scaler.fit(X)
    x = scaler.transform(X)
    kmeans = KMeans(n_clusters=k)
    y_pred = kmeans.fit_predict(x)
    clus_num.append(k)
    inertias.append(kmeans.inertia_)
    
import plotly.graph_objects as go
fig = go.Figure(data=go.Scatter(x=clus_num, y=inertias, mode='markers'))
fig.show()
# 19 clusters it is

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
x = scaler.transform(X)
k = 29
kmeans = KMeans(n_clusters=k)
y_pred = kmeans.fit_predict(x)
clusters = X[['price', 'area', 'rooms']].copy()
clusters['cluster'] = y_pred

In [ ]:
d = d.assign(cluster=y_pred)

In [ ]:
# Assign district numerical value
district_num = {l:k for k,l in enumerate(d.district.unique())}
d['district_num'] = d.district.map(district_num)

In [ ]:
d.head()

In [ ]:
fig =px.scatter(d, x="price_per_m", y="district", color="cluster",
              hover_data=['rooms', 'url', 'item_id', 'website'])
fig.show()

In [ ]:
fig =px.scatter(d, x="latitude_from_center", y="longitude_from_center", color="cluster",
              hover_data=['rooms', 'url', 'item_id', 'website'])
fig.show()

In [ ]:
fig = px.scatter(d, x="rooms", y="cluster", color="owner",
                    hover_data=['rooms', 'url', 'item_id', 'website'])
fig.show()

In [ ]:
pd.options.display.max_rows = 9999
d.groupby(by=['cluster']).agg({'price_per_m': ['mean', 'std', 'count'],
                                                    'price': ['mean', 'std'],
                                                    'rooms': ['median'],
                                                    'latitude': ['mean'],
                                                    'longitude': ['mean']
                                                    })